In [ ]:
import os
# cuBLAS 결정론 모드
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"    # 또는 ":4096:8"
import umap.umap_ as umap

import random, numpy as np, torch
random.seed(2025); np.random.seed(2025); torch.manual_seed(2025)
import scipy.io
import numpy as np
import json
import pandas as pd
import pycwt as cwt
from scipy.signal import iirnotch, filtfilt
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.preprocessing import StandardScaler,minmax_scale
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
np.int = int
from scipy.spatial.distance import pdist, squareform

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# =============================================================
# 하드웨어/재현성 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


# (선택이지만 권장) TF32 비활성화로 추가 일관성
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# =============================================================
# 경로/기본 파라미터
matfilepath=r'D:\Dropbox\Shank2_revision\corrections' #neural/behavioral data of prefrontal cortex in interacting mice
positionfilepath=r"D:\Dropbox\Shank2_revision\PositionData" #AI-infered position data
result=[[] for _ in range(16)]

# =============================================================
# 공용 유틸
def annotation(s1,n1,r1,s2,n2,r2,fs,length):
    timestamp=np.arange(0,length,1/fs)
    result=np.ones((2,len(timestamp)))

    for x in n1:
        mask=(x[0]<timestamp)&(timestamp<x[1])
        result[0][mask]=1*np.ones(np.sum(mask))
    for x in r1:
        mask=(x[0]<timestamp)&(timestamp<x[1])
        result[0][mask]=1*np.ones(np.sum(mask))    
    for x in s1:
        mask=(x[0]<timestamp)&(timestamp<x[1])
        if np.sum(mask)==0:
            result[0][mask]=0*np.ones(np.sum(mask))
        result[0][int(x[0]*fs)]=0
    for x in n2:
        mask=(x[0]<timestamp)&(timestamp<x[1])
        result[1][mask]=1*np.ones(np.sum(mask))
    for x in r2:
        mask=(x[0]<timestamp)&(timestamp<x[1])
        result[1][mask]=1*np.ones(np.sum(mask))  
    for x in s2:
        mask=(x[0]<timestamp)&(timestamp<x[1])
        if np.sum(mask)==0:
            result[1][mask]=0*np.ones(np.sum(mask))
        result[1][int(x[0]*fs)]=0
    return result

def behavioronehot(behavior,totallength,fps,targetfs):
    ts=np.arange(0,totallength,1/fps)
    tempresult=np.zeros((len(behavior),len(ts)))
    for i in range(len(behavior)):
        for j in range(len(behavior[i])):
            s=behavior[i][j][0]
            e=behavior[i][j][1]
            mask=((s<=ts)&(ts<=e))
            tempresult[i][mask]=np.ones(np.sum(mask))
    dsratio=int(fps/targetfs)
    result=np.zeros((len(behavior),int(len(ts)-dsratio)))
    for i in range(len(result[0])):
        result[:,i]=np.mean(tempresult[:,i:dsratio+i],axis=1)
    return result.T
def calpcc(lfp1,lfp2):
    length=np.min((len(lfp1),len(lfp2)))
    lfp1=lfp1[:length]
    lfp2=lfp2[:length] 
    return np.corrcoef(lfp1,lfp2)[0][1]

def cate(behaviorarray):
    result=[x for x in behaviorarray if len(x)>0]    
    result=np.concatenate(result,axis=0)
    return result
class SimpleRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=16,output_size=21):
        super(SimpleRNN, self).__init__()
        self.lg=nn.Linear(input_size,output_size)
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size )  # Regression output

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :]).unsqueeze(1)  # take last time step
      
class sequencedataset(TensorDataset):
    def __init__(self,x,y):
        self.X = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]     
        
def RNN_run_validate(Xtr,Ytr,Xva,Yva,lr,batch_size,hidden_size,seq_len,iter,input_length,output_size):
    model=SimpleRNN(input_length,hidden_size,output_size).to(device)
    criterion=nn.MSELoss()
    optimizer=optim.Adam(model.parameters(),lr=lr, weight_decay=1e-4)
    traindataset=sequencedataset(Xtr,Ytr)
    traindataloader=DataLoader(traindataset,batch_size=batch_size)
    valdataset=sequencedataset(Xva,Yva)
    valdataloader=DataLoader(valdataset,batch_size=batch_size)
    best_loss=np.inf
    best_model=None
    count=0
    tolerance=5
    avg_train_loss_array=[]
    avg_val_loss_array=[]
    avg_best_loss_array=[]
    for epoch in range(iter):
        model.train()
        train_losses=[]
        for batch_x,batch_y in traindataloader:
            batch_x,batch_y=batch_x.to(device),batch_y.to(device)
            optimizer.zero_grad()
            outputs=model(batch_x)
            loss=criterion(outputs,batch_y)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        model.eval()
        val_losses=[]
        with torch.no_grad():
            for batch_x,batch_y in valdataloader:
                batch_x,batch_y=batch_x.to(device),batch_y.to(device)
                val_outputs=model(batch_x)
                val_loss=criterion(val_outputs,batch_y)
                val_losses.append(val_loss.item())
        avg_val_loss=np.mean(val_losses)
        avg_train_loss=np.mean(train_losses)
        avg_train_loss_array.append(avg_train_loss)
        avg_val_loss_array.append(avg_val_loss)
        if epoch%100==0:
            print(f'train loss:{avg_train_loss}, val loss:{avg_val_loss}')
        if avg_val_loss<best_loss:
            best_loss=avg_val_loss
            best_model=model.state_dict()
            avg_best_loss_array.append(best_loss)
            count=0
        else:
            avg_best_loss_array.append(best_loss)
            count+=1
        if count>tolerance:
            break
    plt.plot(np.log(avg_train_loss_array))
    plt.plot(np.log(avg_val_loss_array))
    plt.plot(np.log(avg_best_loss_array))
    plt.legend(['train','val','best'])
    plt.show()
    return best_loss,best_model
def RNN_evaluation(model_state,Xte,Yte,hidden_size,input_length,output_size):
    model=SimpleRNN(input_length,hidden_size,output_size)
    model.load_state_dict(model_state)
    model.eval()
    R2=[]
    with torch.no_grad():
        yhat=model(torch.tensor(Xte,dtype=torch.float32)).squeeze(1).detach().cpu().numpy()
        print(yhat.shape)
        print(Yte.shape)
        for i in range(len(yhat[0])):
            if np.var(yhat[:,i])*np.var(Yte[:,i]):
                R2.append(r2_score(yhat[:,i],Yte[:,i]))
            else:
                R2.append(0)
    return R2
f0 = 60.0
Q = 30.0 
def notch(x,b,a):
    x=x.T
    for i in range(len(x)):
        x[i]=filtfilt(b,a,x[i])
    return np.nanmean(x,axis=0)
def devide_k(n,k):
    length=n//k
    leftover=n%k
    result=[]
    for i in range(k):
        result.append(np.arange(i*length,(i+1)*length+(1 if i<leftover else 0),1))
    return result
def create_seq(index_list,seq_len):
    result=[]
    for index in index_list:
        edited_index=index[int(seq_len/2):-int(seq_len/2)]
        for i in edited_index:
            result.append(np.arange(i-int(seq_len/2),i+int(seq_len/2)))
    return result
from sklearn.metrics import accuracy_score, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import silhouette_score
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
def nested_cv_rnn_raw(X,Y,*,outer_k=5,lr=0.0001,batch_size=128,hidden_size=64,seq_len=50,iter=100):
    X=np.array(X)
    Y=np.array(Y)
    n_samples=len(X)
    result=[]
    input_length=X.shape[-1]
    output_size=Y.shape[-1]
    indexlist=devide_k(n_samples,outer_k)
    inner_k=outer_k-1
    for iii in range(outer_k):
        testindex=[indexlist[iii]]
        innerindex=[]
        for i in range(outer_k):
            if i!=iii:
                innerindex.append(indexlist[i])
        total_best_loss=np.inf
        total_best_model=None
        for ii in range(inner_k):
            valindex=[innerindex[ii]]
            trainindex=[]
            for i in range(inner_k):
                if i!=ii:
                    trainindex.append(innerindex[i])
            trainseq=create_seq(trainindex,seq_len)
            valseq=create_seq(valindex,seq_len)
            X_tr=np.array([X[seq] for seq in trainseq])
            Y_tr=np.array([(Y[seq])[-1] for seq in trainseq])
            X_va=np.array([X[seq] for seq in valseq])
            Y_va=np.array([(Y[seq])[-1] for seq in valseq])
            scX=StandardScaler().fit(X_tr.reshape(-1,(X_tr).shape[-1]))
            scY=StandardScaler().fit(Y_tr.reshape(-1,(Y_tr).shape[-1]))
            Xtr=scX.transform(X_tr.reshape(-1,X_tr.shape[-1])).reshape(X_tr.shape)
            Ytr=scY.transform(Y_tr.reshape(-1,Y_tr.shape[-1])).reshape(Y_tr.shape)
            Xva=scX.transform(X_va.reshape(-1,X_va.shape[-1])).reshape(X_va.shape)
            Yva=scY.transform(Y_va.reshape(-1,Y_va.shape[-1])).reshape(Y_va.shape)
            best_loss,best_model=RNN_run_validate(Xtr,Ytr,Xva,Yva,lr=lr,batch_size=batch_size,hidden_size=hidden_size,seq_len=seq_len,iter=iter,input_length=input_length,output_size=output_size)
            if best_loss<total_best_loss:
                total_best_loss=best_loss
                total_best_model=best_model
        trainvalseq=create_seq(innerindex,seq_len)
        testseq=create_seq(testindex,seq_len)
        X_tv=np.array([X[seq] for seq in trainvalseq])
        Y_tv=np.array([(Y[seq])[-1] for seq in trainvalseq])
        X_te=np.array([X[seq] for seq in testseq])
        Y_te=np.array([(Y[seq])[-1] for seq in testseq])    
        scX=StandardScaler().fit(X_tv.reshape(-1,(X_tv).shape[-1]))
        scY=StandardScaler().fit(Y_tv.reshape(-1,(Y_tv).shape[-1]))
        Xtv=scX.transform(X_tv.reshape(-1,X_tv.shape[-1])).reshape(X_tv.shape)
        Ytv=scY.transform(Y_tv.reshape(-1,Y_tv.shape[-1])).reshape(Y_tv.shape)
        Xte=scX.transform(X_te.reshape(-1,X_te.shape[-1])).reshape(X_te.shape)
        Yte=scY.transform(Y_te.reshape(-1,Y_te.shape[-1])).reshape(Y_te.shape)
        result.append(RNN_evaluation(total_best_model,Xte,Yte,hidden_size,input_length,output_size))
    return np.mean(result)
def slidingwindowfft(wave,dt,finalfreq):
    ratio=int(1/(dt*finalfreq))
    freq=np.fft.fftfreq(ratio,dt)
    length=int(len(wave)/ratio)
    lfp=np.zeros((len(freq),length),dtype=np.complex128)
    for i in range(length):
        lfp[:,i]=np.fft.fft(wave[i*ratio:(i+1)*ratio])    
    return lfp,freq
finalfreq=1

result=[[],[],[],[]]
for file in os.listdir(matfilepath):
    if (not file.endswith('LFP.mat')) & (file.endswith('.mat')):
#loading LFP(Local Field Potential) data, 2000Hz, 10min
        Kfilepath=os.path.join(matfilepath,file)
        LFPfilepath=Kfilepath[:-4]+'_LFP.mat'
        if not os.path.exists(LFPfilepath):
            continue
        else:
            LFP=scipy.io.loadmat(LFPfilepath)
        fs=LFP['SamplingFreq'][0][0]
        lfpraw1=LFP['raw_signal_final'][0][0]

        lfpraw2=LFP['raw_signal_final'][0][1]
        

#loading genotype data: WT/KO(autism model mouse)
        K=scipy.io.loadmat(Kfilepath)['K']
        ID1=file[:4]+K['ID'][0][0][0][0][0]
        ID2=file[:4]+K['ID'][0][0][0][1][0]
        if K['ID'][0][0][0][0][0].endswith('WT'):
            type1=0
        else:
            type1=1
        if K['ID'][0][0][0][1][0].endswith('WT'):
            type2=0
        else:
            type2=1
        if K['ID'][0][0][0][0][0].endswith('WT'):
            if type1+type2==1:
                m1=1
            else:
                m1=0
        else:
            if type1+type2==1:
                m1=2
            else:
                m1=3
        if K['ID'][0][0][0][1][0].endswith('WT'):
            if type1+type2==1:
                m2=1
            else:
                m2=0
        else:
            if type1+type2==1:
                m2=2
            else:
                m2=3
        
#loading position data: inferred using DEEPLABCUT, 30Hz, for 2 mice
#normalizing, differentiating, and simple mathmatical calculation to achieve kinematic features
        csvfilepath=os.path.join(positionfilepath,file[:-4]+'.csv')
        if os.path.exists(csvfilepath):
            positiondata1=pd.read_csv(csvfilepath)[['Center_1_x','Center_1_y']].to_numpy()
            positiondata2=pd.read_csv(csvfilepath)[['Center_2_x','Center_2_y']].to_numpy()
            xx1=minmax_scale(positiondata1[:,0])
            yy1=minmax_scale(positiondata1[:,1])
            xx2=minmax_scale(positiondata2[:,0])
            yy2=minmax_scale(positiondata2[:,1])
            positiondata1[:,0]=xx1
            positiondata1[:,1]=yy1
            positiondata2[:,0]=xx2
            positiondata2[:,1]=yy2
            distance=np.sqrt(np.sum((positiondata1-positiondata2)**2,axis=1))
            vvdis=distance[1:]-distance[:-1]
            vvx1=xx1[1:]-xx1[:-1]
            vvx2=xx2[1:]-xx2[:-1]
            vvy1=yy1[1:]-yy1[:-1]
            vvy2=yy2[1:]-yy2[:-1]
            displacement1=positiondata1[1:,:]-positiondata1[:-1,:]
            velocity1=np.sqrt(displacement1[:,0]**2+displacement1[:,1]**2)
            displacement2=positiondata2[1:,:]-positiondata2[:-1,:]
            velocity2=np.sqrt(displacement2[:,0]**2+displacement2[:,1]**2)
            dsrate=int(30/finalfreq)
            x1=np.zeros(int((len(velocity1)-dsrate)))
            x2=np.zeros(int((len(velocity1)-dsrate)))
            y1=np.zeros(int((len(velocity1)-dsrate)))
            y2=np.zeros(int((len(velocity1)-dsrate)))
            v1=np.zeros(int((len(velocity1)-dsrate)))
            vx1=np.zeros(int((len(velocity1)-dsrate)))
            vx2=np.zeros(int((len(velocity1)-dsrate)))
            vy1=np.zeros(int((len(velocity1)-dsrate)))
            vy2=np.zeros(int((len(velocity1)-dsrate)))
            vdis=np.zeros(int((len(velocity1)-dsrate)))
            v2=np.zeros(int((len(velocity1)-dsrate)))
            dis=np.zeros(int((len(velocity1)-dsrate)))
            
            for i in range(len(v1)):
                x1[i]=np.mean(xx1[i:i+dsrate])
                y1[i]=np.mean(yy1[i:i+dsrate])
                x2[i]=np.mean(xx2[i:i+dsrate])
                y2[i]=np.mean(yy2[i:i+dsrate])
                v1[i]=np.mean(velocity1[i:i+dsrate])
                v2[i]=np.mean(velocity2[i:i+dsrate])
                dis[i]=np.mean(distance[i:i+dsrate])
                vdis[i]=np.mean(vvdis[i:i+dsrate])
                vx1[i]=np.mean(vvx1[i:i+dsrate])
                vy1[i]=np.mean(vvy1[i:i+dsrate])
                vx2[i]=np.mean(vvx2[i:i+dsrate])
                vy2[i]=np.mean(vvy2[i:i+dsrate])
        else:
            continue
        
        cx1=x1-0.5
        cx2=x2-0.5
        cy1=y1-0.5
        cy2=y2-0.5
        c1=np.sqrt(cx1**2+cy1**2)
        c2=np.sqrt(cx2**2+cy2**2)
#loading manualing annotated data: 30Hz, 10min
#grouped as social/nonsocial/rest behavior
        s1behavior=[np.array(K['SocialBehavior'][0][0][0][1][0][0]),np.array(K['SocialBehavior'][0][0][0][1][0][1]),np.array(K['SocialBehavior'][0][0][0][1][0][2]),np.array(K['SocialBehavior'][0][0][0][1][0][3])]
        s1behavior=[x for x in s1behavior if len(x)]
        selfsocialbehavior=np.concatenate(s1behavior,axis=0)
        ns1behavior=[np.array(K['NonSocialBehavior'][0][0][0][1][0][0]),np.array(K['NonSocialBehavior'][0][0][0][1][0][1]),np.array(K['NonSocialBehavior'][0][0][0][1][0][2]),np.array(K['Exploring'][0][0][0][1][0][0]),np.array(K['Exploring'][0][0][0][1][0][0]),np.array(K['Exploring'][0][0][0][1][0][1]),np.array(K['Exploring'][0][0][0][1][0][2])]
        ns1behavior=[x for x in ns1behavior if len(x)>0]
        selfnonsocialbehavior=np.concatenate(ns1behavior,axis=0)
        s2behavior=[np.array(K['SocialBehavior'][0][0][0][2][0][0]),np.array(K['SocialBehavior'][0][0][0][2][0][1]),np.array(K['SocialBehavior'][0][0][0][2][0][2]),np.array(K['SocialBehavior'][0][0][0][2][0][3])]
        s2behavior=[x for x in s2behavior if len(x)>0]
        othersocialbehavior=np.concatenate(s2behavior,axis=0)
        ns2behavior=[np.array(K['NonSocialBehavior'][0][0][0][2][0][0]),np.array(K['NonSocialBehavior'][0][0][0][2][0][1]),np.array(K['NonSocialBehavior'][0][0][0][2][0][2]),np.array(K['Exploring'][0][0][0][2][0][0]),np.array(K['Exploring'][0][0][0][2][0][0]),np.array(K['Exploring'][0][0][0][2][0][1]),np.array(K['Exploring'][0][0][0][2][0][2])]
        ns2behavior=[x for x in ns2behavior if len(x)>0]
        othernonsocialbehavior=np.concatenate(ns2behavior,axis=0)
        otherrestbehavior=np.array(K['Immobile'][0][0][0][2][0][0]) if len(K['Immobile'][0][0][0][2][0][0])>0 else np.zeros((1,3))
        selfrestbehavior=np.array(K['Immobile'][0][0][0][1][0][0]) if len(K['Immobile'][0][0][0][1][0][0])>0 else np.zeros((1,3))   
#validating that the recording and analysis has been done for 10mins
        if len(K['Event'][0][0][0][3])>1:
            starttime=K['Event'][0][0][0][3][0]
            endtime=K['Event'][0][0][0][3][1]
        else:
            starttime=K['Event'][0][0][0][3][0][0]
            endtime=K['Event'][0][0][0][3][0][1]   
        totaltime=float((endtime-starttime)/(1e6))
        totaltime=np.min([totaltime,600])

#loading spike(neuron activity timestamps data)->converting into firing rate(how many activities per time bin) data   
        SPK=K['SPK'][0][0]
        num=0
        s1=[]
        s2=[]
        for iii in range(len(SPK)):
            spike=SPK[iii][0]
            if len(spike)>1:
                spike=spike-starttime/1e6
                ts=np.arange(0,totaltime,1/30)
                if np.sum((spike>0)&(spike<totaltime))/totaltime >0.5:
                    temp=[]
                    for i in range(len(ts)-1):
                        temp.append(np.sum((ts[i]<=spike)&(spike<ts[i]+1/finalfreq))*finalfreq)
                    s1.append(temp/np.mean(temp))
            spike=SPK[iii][1]
            if len(spike)>1:
                spike=spike-starttime/1e6
                ts=np.arange(0,totaltime,1/30)
                if np.sum((spike>0)&(spike<totaltime))/totaltime >0.5:
                    temp=[]
                    for i in range(len(ts)-1):
                        temp.append(np.sum((ts[i]<=spike)&(spike<ts[i]+1/finalfreq))*finalfreq)
                    s2.append(temp/np.mean(temp))
#checkpoint: the number of neuron recorded varies session by session, so insures the presence of neurons in both mice        
        if (len(s1)>1)*(len(s2)>1)==0:
            continue

        s1=np.array(s1).T
        s2=np.array(s2).T
#convert LFP raw trace->LFP wavelet power(square of instaneous amplitude) trace for different bands
        dt = 1/fs  # 샘플링 간격 (s)
        t = np.arange(0, totaltime, dt)  # 10초

        # 2. Morlet wavelet transform
        mother = cwt.wavelet.Morlet(6)  # Morlet wavelet, w0=6
        s0 = 2 * dt                    # 최소 scale
        dj = 1/12                       # scale step
        J = 7 / dj                      # scale 개수

        wave1, scales, freqs, coi, fft, fftfreqs = cwt.cwt(lfpraw1, dt, dj, s0, J, mother)   
        wave2, scales, freqs, coi, fft, fftfreqs = cwt.cwt(lfpraw2, dt, dj, s0, J, mother)      
        # Wavelet → 밴드 파워 → 10Hz 다운샘플 → 표준화
        # wave1,wfreqs=slidingwindowfft(lfpraw1,1/fs,finalfreq)
        # wave2,wfreqs=slidingwindowfft(lfpraw2,1/fs,finalfreq)   
        # # wave1=artifactcorrection(wave1)
        # # wave2=artifactcorrection(wave2)
        band=[[4,12],[13,30],[31,80],[4,100]]
        lfp1=[[],[],[],[]]
        lfp2=[[],[],[],[]]

        for i in range(4):
            freqmask=(band[i][0]<=freqs)&(band[i][1]>=freqs)
            lfp1[i]=(np.sum(np.abs(wave1[freqmask])**2,axis=0)).flatten()
            lfp2[i]=(np.sum(np.abs(wave2[freqmask])**2,axis=0)).flatten()

        lfp1=np.array(lfp1)/np.max(lfp1)
        lfp2=np.array(lfp2)/np.max(lfp2)
        lfp1=lfp1.T
        lfp2=lfp2.T
        lfp1ds=np.zeros((int((len(lfp1)-int(fs/finalfreq))),4))
        lfp2ds=np.zeros((int((len(lfp2)-int(fs/finalfreq))),4))
        for i in range(len(lfp1ds)):
            lfp1ds[i,:]=np.mean(lfp1[i:i+int(fs/finalfreq),:],axis=0)
            lfp2ds[i,:]=np.mean(lfp2[i:i+int(fs/finalfreq),:],axis=0)
#onehotencoding of mannually annotated behavior data
        b1=behavioronehot([selfsocialbehavior,selfnonsocialbehavior,selfrestbehavior],totaltime,30,finalfreq)
        b2=behavioronehot([othersocialbehavior,othernonsocialbehavior,otherrestbehavior],totaltime,30,finalfreq)


        minlen=np.min([len(v1),len(v2),len(b1),len(b2),len(dis),len(x1),len(x2),len(y1),len(y2),len(s1),len(s2),len(lfp1ds),len(lfp2ds)])
#final scaling and fragmenting time-serious position data
        x1=(x1[:minlen].reshape(-1,1))
        x2=(x2[:minlen].reshape(-1,1))
        y1=(y1[:minlen].reshape(-1,1))
        y2=(y2[:minlen].reshape(-1,1))
        v1=(85*(v1[:minlen].reshape(-1,1)))
        v2=(85*(v2[:minlen].reshape(-1,1)))
        dis=(dis[:minlen].reshape(-1,1))
        time=np.linspace(0,1,minlen).reshape(-1,1)
        b1=b1[:minlen]
        b2=b2[:minlen]
        s1=s1[:minlen]
        s2=s2[:minlen]
        lfp1ds=minmax_scale(lfp1ds[:minlen])
        lfp2ds=minmax_scale(lfp2ds[:minlen])
        vx1=85*vx1[:minlen].reshape(-1,1)
        vx2=85*vx2[:minlen].reshape(-1,1)
        vy1=85*vy1[:minlen].reshape(-1,1)
        vy2=85*vy2[:minlen].reshape(-1,1)
        vdis=85*vdis[:minlen].reshape(-1,1)
        c1=3.1*(c1[:minlen].reshape(-1,1))
        c2=3.1*(c2[:minlen].reshape(-1,1))
        seq_len = 10
        outer_k = 5

#concatenating all data into single data matrix, excuting training/validation/test
        Y_raw=np.concatenate([x1,x2,y1,y2,v1,v2,dis,vx1,vx2,vy1,vy2,vdis,c1,c2,lfp1ds,lfp2ds],axis=1)
        name_dic=np.array(['x1','x2','y1','y2','v1','v2','dis','vx1','vx2','vy1','vy2','vdis','c1','c2','1theta','1beta','1gamma','1wide','2theta','2beta','2gamma','2wide'])
        try:
            X_raw=np.log(s1)
            mean_r2=nested_cv_rnn_raw(X_raw,Y_raw,outer_k=outer_k,lr=1e-4,batch_size=32,hidden_size=32,seq_len=2,iter=1000)
            mean_r2=np.array(mean_r2)
            print(mean_r2)
            print(name_dic[mean_r2>0])
            print(mean_r2[mean_r2>0])

            X_raw=np.log(s2)
            mean_r2=nested_cv_rnn_raw(X_raw,Y_raw,outer_k=outer_k,lr=1e-4,batch_size=32,hidden_size=32,seq_len=2,iter=1000)
            mean_r2=np.array(mean_r2)
            print(mean_r2)
            print(name_dic[mean_r2>0])
            print(mean_r2[mean_r2>0])
        except Exception as e:
            print(e)
            continue
pd.DataFrame(result).to_csv("R2_RNN_LFP_nestedCV.csv")        



